# Chapter7 앙상블 학습 & 랜덤 포레스트

집단지성처럼 한 예측을 수행할 때 여러 예측기를 통해 예측을 수집해서 가장 좋은 모델을 선택하는 방법을 앙상블 방법이라고 한다. 이런 방법을 트리에 사용해 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 여러가지 트리를 생성해 가장 많은 선택을 받은 클래스를 예측으로 삼는 학습알고리즘을 랜덤포레스트라고 한다.

## 1. 투표기반 분류기
로지스틱 회기 분류기, SVM 분류기, 랜덤포레스트 분류기, K-nearest 분류기 등등을 사용해 한 데이터의 클래스를 예측하고 가장 많은 선택(투표)를 받은 클래스를 선정하는 것을 직접투표(hard voting)이라고 한다. (다수결 투표방식)

이런 다수결 투표방식은 각 예측기가 가능한 한 서로 독립적일때 최고의 성능을 발휘한다. 모든 예측기가 같은 종류의 오차를 만들어낸다면 잘못된 클래스가 다수결 투표에 의해 선택될 수 있기 때문이다.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression() #로지스틱회귀
rnd_clf = RandomForestClassifier() #랜덤포레스트분류기
svm_clf = SVC() #SVM분류기

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [6]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.904


각 개개인의 분류기보다 조금 더 나은 성능 (0.904)를 보이는걸 확인할 수 있다.

### 간접투표란?
개별의 분류기들이 각각 클래스를 그냥 선정하는 것이 아니라, 각 데이터마다 클래스에 속할 확률을 예측하는 기능을 수행할 수 있으면, (predict_proba()메소드가 있다면) 개별 분류기의 예측의 평균을 내서 가장 높은 클래스를 예측할 수 있고 이를 간접투표(soft voting)이라 한다.

### 배깅과 페이스팅이란?
앞서 말한것은 여러 분류기를 사용해 같은 데이터를 다른 훈련 알고리즘을 사용하는 것인데, 같은 분류기를 사용하더라도 다른 훈련세트를 적용하여 (훈련데이터의 서브셋을 무작위로 선정) 같은 분류기를 다르게 학습시키는 것이다.
이때, 훈련 세트에서 중복을 허용하여 샘플링하는 방식을 배깅(bagging, boostrap aggregating의 줄임말)이라 하고
중복을 허용하지 않고 샘플링 하는 방시을 페이스팅(pasting)이라 한다.
즉, 한 예측기에 같은 훈렴 샘플이 여러개 들어갈 수 있다면 배깅 방식이고 서로 다른 훈련 샘플만 들어간다면 페이스팅 방식이다.

## 사이킷런의 배깅 & 페이스팅
사이킷런은 이들을 위해 BagginClassifier(회귀의 경우 BaggingRegressor)를 제공한다.

In [9]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators = 500,
max_samples=100, bootstrap=True, n_jobs = -1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.912


In [11]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

0.856


부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높다.
일반적으로는 배깅이 좀 더 나은 모델을 만들기 때문에 페이스팅보다 좀 더 선호한다.

### oob평가란?
배깅을 사용하면 한 예측기를 위해 같은 샘플이 여러번 샘플링 될 수 있고, 어떤 샘플은 아예 선택되지 않을 수 있다. 이 남겨진 샘플은 각 예측기마다 다르다. 즉, 한 예측기의 성능을 평가할 때 이 oob(out-of-bag)샘플을 사용할 수 있으므로 별도의 검증 세트를 사용하지 않을 수 있다. 앙상블의 평가는 각 예측기의 oob평가를 평균하여 얻는다.
BaggingClassifier에서 oob_score = True로 설정하면 훈련이 끝난 후 자동으로 oob평가를 수행한다. 평가 점수는 oob_score_변수에 저장돼있다.

In [12]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, oob_score=True, random_state=40)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.8986666666666666

위 코드는 oob평가를 사용해 BaggingClassifier가 테스트셋에서 89%정도의 정확도를 얻었다고 알 수 있다. 그렇다면 이제 실제 테스트셋을 사용해 평가해보면 어떨까?

In [13]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.912

테스트셋을 사용해 평가한 경우 91.2%정도의 정확도를 보이고 있으므로 유사하다고 말할 수 있다.

## 랜덤포레스트
랜덤포레스트는 일반적으로 배깅(혹은 페이스팅)을 적용한 결정 트리의 앙상블이다.
<strong>max_samples</strong>로 훈련 세트의 크기를 지정한다.

In [14]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

랜덤포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입한다.
이는 결국 트리를 더욱 다양하게 만들어 편향을 손해보고 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어낸다.

## 엑스트라 트리
앞에서 설명한거처럼 무작위성을 극도로 끌어올린 무작위한 트리의 랜덤 포레스트를 익스트림 랜덤 트리 앙상블 이라고 부른다.
모든 노드에서 특성마다 가장 최적의 임곗값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요되는 작업 중 하나이므로 일반적인 랜덤포레스트보다 엑스트라 트리가 훨씬 빠르다.

# 특성 중요도
특성 중요도란 어떤 특성이 다른 특성보다 데이터를 분류하는데 더 중요하냐를 말하는 것인데 바꿔말하면 불순도를 더 많이 감소시킬 수 있는 특성이 중요도가 높다는 것이다.
사이킷런은 이런 중요도 측정을 지원하고 <strong> feature_importances_ </strong>변수에 저장한다.

In [16]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.11249225099876375
sepal width (cm) 0.02311928828251033
petal length (cm) 0.4410304643639577
petal width (cm) 0.4233579963547682


# 부스팅
부스팅(boosting)이란 약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블 방법을 말한다.
앞의 모델을 보완해나가면서 일련의 예측기를 학습시키는 것이다. 부스팅 방법에는 여러가지가 있지만
에이다부스트(AdaBoost, adaptive boosting의 줄임말)과
그라디언트 부스팅(gradient boosting)이 인기가 많다.

## 1.에이다부스트
에이다부스트는 첫 분류기가 훈련세트를 통해 훈련시키고 예측을 만든다 -> 잘못 분류된 샘플의 가중치를 상대적으로 높여서 다음 분류기에게 전달한다 -> 반복 으로 진행된다. 처음엔 m개의 샘플의 가중치는 모두 1/m으로 동일하다.
사이킷런은 SAMME라는 에이다부스트의 다중 클래스 버전을 사용한다. 클래스가 두개 뿐일때는 SAMME가 에이다 부스트와 동일하다. 만약 <strong> 예측기가 클래스의 확률을 추정할 수 있다면(predict_proba()메소드가 있다면)</strong> 사이킷런은 SAMME.R(R은 Real을 말한다)라는 SAMME의 변정을 사용한다.
SAMME : stagewise additive modeling using a multiclass exponential loss function

In [17]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200, random_state=42)

## 2.그라디언트 부스팅
그라디언트 부스팅은 에이다부스트처럼 순차적으로 예측기들을 연결하지만 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 것이 아닌 이전 예측기가 만든 <strong>잔여 오차(residual error)</strong>에 새로운 예측기를 학습시킨다. 결정트리를 기반 에측기로 사용하는 회귀모델은 그라디언트 트리 부스팅 혹은 그라디언트 부스티드 회귀 트리(GBRT) 라고 한다.

In [19]:
#임의의 데이터셋 만들기
import numpy as np
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [20]:
#만든 데이터셋에 결정 트리 회귀 모델을 훈련
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [21]:
#첫번째 예측기가 만든 잔여오차에 두번째 회귀 모델을 훈련
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [22]:
#두번째 예측기가 만든 잔여오차에 세번째 회귀 모델을 훈련
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=42)

#### 이제 세개의 트리를 포함하는 앙상블 모델이 생겼다. 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 된다.

In [23]:
X_new = np.array([[0.8]])
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))
y_pred

array([0.73982889])

위처럼 y1, y2, y3, 처럼 귀찮게 정의하지말고 사이킷런의 <strong> GradientBoostingRegressor</strong>을 사용하면 GBRT앙상블을 간단하게 훈련시킬 수 있다. 트리 수(n_estimators)와 같이 훈련을 제어하는 매개변수도 있고 여러 옵션이 있다.

In [24]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate = 1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

### GBRT의 learning rate이란?
각 트리의 기여 정도를 말한다. 즉 learning rate = 0.1처럼 낮다면 많은 트리가 필요할 것이지만 성능은 올라갈 것이다. 이렇게 learning rate를 낮추는 것은 규제의 일종으로 <strong>축소 shrinkage</strong>라고 부른다.

최적의 트리 수를 찾기 위해 조기종료기법을 사용할 수 있다. <strong>staged_predict()</strong>메소드를 사용한다. 이 메소드는 훈련의 각 단계에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자를 반환한다. 다음 코드는 120개의 트리로 GBRT 앙상블을 훈련시키고 최적의 트리 수를 찾기 위해 각 단계에서 검증 오차를 수행한다.
과정이 종료되면 최적의 트리 수를 사용해 새로운 GBRT 앙상블을 훈련시킨다

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=39, random_state=42)

## 확률적 그라디언트 부스팅이란?
GBRT는 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있는 <strong>subsample</strong>매개변수를 지원한다. 예를 들어 subsample=0.25라고 하면 각 트리는 무작위로 선택된 25%의 훈련샘플로 학습한다.

# 스태킹
앞에서 설명한 앙상블 모델은 앙상블에 속한 모든 예측기의 예측을 hard voting, soft voting등의 간단한 함수를 사용하여 취합한다. 스태킹은 그런 방법이 아닌 각 예측기가 예측한 값을 마지막으로 하나의 예측기(블렌더 또는 메타학습기라고 부름)에 이 예측을 입력해 마지막 예측기가 최종 예측을 만드는 것이다.
블렌더는 홀드아웃(hold-out)세트를 사용해 학습시킨다.
1. 훈련 세트를 두개의 서브셋으로 나눈다. 첫번째 서브셋은 첫번째 레이어의 예측을 훈련시키기 위해 사용한다
2. 첫번째 레이어의 예측기를 사용해 두번째 (홀드아웃)세트에 대한 예측을 만든다. 예측기들이 훈련하는 동안 두번째 샘플(홀드아웃)세트는 전혀 보지 못했기 때문에 이때 만들어지는 예측은 완전히 새로운 것이다.
3. 이제 홀드아웃 세트의 각 샘플에 대해 세 개의 예측값이 있다. (새로운 훈련세트는 3차원이다.) 블렌더가 이 새로운 훈련 세트로 훈련한다. 즉, 첫번째 레이어의 예측을 가지고 타깃값을 예측하도록 학습한다.